In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -qU bitsandbytes evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [3]:
# Setup the Seed and the model name to be used
SEED=12345
MODEL_NAME = 'distilbert-base-uncased'

In [4]:
# Import the libraries
from datasets import load_dataset
import numpy as np
from transformers  import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
import torch


from sklearn.metrics import accuracy_score

# Set Seed
np.random.seed(SEED)
torch.manual_seed(SEED)

In [5]:
# Load the IMDB data set and split it into training and testing
# Test Size = 20%
dataset = load_dataset('imdb',
                       split='train'
                      ).train_test_split(test_size=0.2,
                                         shuffle=True,
                                         seed=SEED
                                        )

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
dataset['train'][0].get('text'), dataset['train'][0].get('label')

("Henry Thomas showed a restraint, even when the third act turned into horrible hollywood resolution that could've killed this movie, that kept the dignity of a redemption story and as for pure creepiness-sniffing babies?",
 1)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [8]:
print(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

distilbert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
#Tokenize the training and test dataset
tokenized_dataset = {}

tokenized_dataset['train'] = dataset['train'].map(lambda x: tokenizer(x['text'],
                                                                      truncation=True
                                                                     ),
                                                  batched = True
                                                 )
tokenized_dataset['test'] = dataset['test'].map(lambda x: tokenizer(x['text'],
                                                                    truncation=True
                                                                   ),
                                                batched = True
                                               )

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset

{'train': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 20000
 }),
 'test': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 5000
 })}

In [11]:
tokenized_dataset['train'][0]['text']

"Henry Thomas showed a restraint, even when the third act turned into horrible hollywood resolution that could've killed this movie, that kept the dignity of a redemption story and as for pure creepiness-sniffing babies?"

In [12]:
LEARNING_RATE = 0.001
BATCH_SIZE = 32
NUM_EPOCHS = 2
WEIGHT_DECAY = 0.01

In [13]:
# Define the function to compute the accuracy
def compute_metrics(eval_pred):
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [14]:
# Define training arguments
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_args = TrainingArguments(output_dir = './peft_lora',
                               learning_rate = LEARNING_RATE,
                               per_device_train_batch_size=BATCH_SIZE,
                               per_device_eval_batch_size=BATCH_SIZE,
                               #evaluation_strategy='epoch',
                               eval_strategy = 'epoch',
                               save_strategy='epoch',
                               num_train_epochs=NUM_EPOCHS,
                               report_to = 'none',
                               weight_decay=WEIGHT_DECAY,
                               load_best_model_at_end=True
                              )

In [15]:
# Create the original model
omodel = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                            num_labels=2,
                                                            force_download=True
                                                           )

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
omodel

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [17]:
# Define a LoraConfig. The task type is Sequence Classification, r=8 (as per the documentation),
# target modules for the MODEL are q_lin, k_lin
# Set the Lora Alpha parameter to 32 and the dropout to 0.1
# https://huggingface.co/docs/peft/en/package_reference/lora
lora_config = LoraConfig(task_type = TaskType.SEQ_CLS,
                         inference_mode = False,
                         r=8,
                         target_modules= ['q_lin','k_lin'],
                         lora_alpha=32,
                         lora_dropout=0.1
                        )

In [18]:
# Get the lora model by passing the config defined above
lora_model = get_peft_model(omodel,
                            lora_config
                           )

In [19]:
lora_model.print_trainable_parameters()

trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [20]:
# Define Lora Trainer
ltrainer = Trainer(model = lora_model,
                   args = train_args,
                   train_dataset = tokenized_dataset['train'],
                   eval_dataset = tokenized_dataset['test'],
                   tokenizer=tokenizer,
                   data_collator = data_collator,
                   compute_metrics = compute_metrics
                  )

In [21]:
# Now we train the model
ltrainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.280200,0.232083,0.909400
2,0.215400,0.224238,0.916800


TrainOutput(global_step=1250, training_loss=0.23470531311035156, metrics={'train_runtime': 1569.6074, 'train_samples_per_second': 25.484, 'train_steps_per_second': 0.796, 'total_flos': 5386721482737408.0, 'train_loss': 0.23470531311035156, 'epoch': 2.0})

In [22]:
# Evaluate the lora trainer
ltrainer.evaluate()

{'eval_loss': 0.22423788905143738,
 'eval_accuracy': 0.9168,
 'eval_runtime': 76.6742,
 'eval_samples_per_second': 65.211,
 'eval_steps_per_second': 2.048,
 'epoch': 2.0}

In [23]:
# Randomly pickup samples from the Test Dataset
n_test_samples = 50
test_samples = np.random.randint(0, dataset['test'].num_rows, n_test_samples)

tokenized_samples = tokenized_dataset['test'].select(test_samples)

In [24]:
# Use the base model to predict the outcome
results = ltrainer.predict(tokenized_samples)

In [25]:
# Prepare the y_pred and y_true
y_pred = results.label_ids

y_true = []
for i in test_samples:
    y_true.append(dataset['test'][int(i)].get('label'))

In [26]:
# Calculate and print the accuracy
accuracy = np.round(accuracy_score(y_true=y_true, y_pred = y_pred)*100,2)
print(f"Accuracy {accuracy}%")

Accuracy 100.0%
